In [1]:
# Allow access to parent directory
import sys
import os
parent_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(1, parent_path)

In [2]:
from helper import download_dataset, download_article
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRReader, DPRQuestionEncoderTokenizerFast, DPRContextEncoderTokenizerFast, DPRReaderTokenizerFast
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.preprocessing import OneHotEncoder

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-11-03 18:57:42.378692: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 18:57:43.775666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-11-03 18:57:46.582216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-03 18:57:46.942407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-03 18:57:46.942461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# Set variables
DATASET_FP = "../WatClaimCheck_dataset" # CHANGE TO MATCH LOCAL
max_seq_len = 128

In [4]:
# Retrieve dataset
train_df, valid_df, test_df = download_dataset(DATASET_FP)

Extracting fields from metadata
Extracting fields from label
Extracting fields from metadata
Extracting fields from label
Extracting fields from metadata
Extracting fields from label


In [12]:
def download_article_as_string(dataset_fp, json_list):
    return '\n'.join([' '.join(download_article(DATASET_FP, json_file)) for json_file in json_list])

train_df['premise_articles_json'] = train_df['premise_articles'].apply(lambda x: x.values())
train_df['premise_articles_content'] = train_df['premise_articles_json'].apply(lambda x: download_article_as_string(DATASET_FP, x))

: 

In [7]:
# DPR
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
reader_tokenizer =  DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')

# train_context_encodings = context_tokenizer(list(train_df['premise_articles_content']), truncation=True, padding="max_length", max_length=max_seq_len, return_tensors='tf')
train_context_encodings = context_tokenizer(list(train_df['premise_articles_content']), return_tensors='tf')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRReaderTokenizer'.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/sysung98/MIDS/W266-final-project/envs/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_8411/1898122481.py", line 7, in <module>
    train_context_encodings = context_tokenizer(list(train_df['premise_articles_content']), return_tensors='tf')
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sysung98/MIDS/W266-final-project/envs/lib/python3.11/site-packages/transformers/tokenization_utils_base.py", line 2790, in __call__
    encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sysung98/MIDS/W266-final-project/envs/lib/python3.11/site-packages/transformers/tokenization_utils_base.py", line 2876, in _call_one
    return self.batch_encode_plus(


In [5]:
def create_multiclass_distilbert_model(
    base_model_name: str
):
    base_model = TFRobertaForSequenceClassification.from_pretrained(base_model_name)
    return base_model

In [7]:
# Get tokenizer and model
base_model_name = "roberta-base"
tokenizer = DistilBertTokenizer.from_pretrained(base_model_name)
model = create_multiclass_distilbert_model(base_model_name)

2023-10-31 11:47:44.331990: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 11:47:44.332172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 11:47:44.332226: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 11:47:50.999089: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 11:47:50.999177: I tensorflow/compile